In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys

import pandas as pd
import numpy as np

import time

import torch

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
sys.path.append(os.path.abspath('./libraries/Unlearnable-Examples/'))
import toolbox

In [5]:
# https://github.com/HanxunH/Unlearnable-Examples/blob/main/perturbation.py#L45

EPSILON = 8
NUM_STEPS = 1
STEP_SIZE = 0.8

In [6]:
# https://github.com/HanxunH/Unlearnable-Examples/blob/main/perturbation.py#L51
# https://github.com/HanxunH/Unlearnable-Examples/blob/main/perturbation.py#L446

noise_generator = toolbox.PerturbationTool(
    epsilon=EPSILON / 255,
    num_steps=NUM_STEPS,
    step_size=STEP_SIZE / 255
)

In [ ]:
def add_random_noise(x):
    
    # https://github.com/HanxunH/Unlearnable-Examples/blob/main/dataset.py#L441
    noise = noise_generator.random_noise(noise_shape=x.shape)
    
    # https://github.com/HanxunH/Unlearnable-Examples/blob/main/dataset.py#L448
    noise = noise.mul(255).clamp_(0, 255)
    
    # https://github.com/HanxunH/Unlearnable-Examples/blob/main/perturbation.py#L451
    x = x + noise.to(x.device)
    x = torch.clamp(x, 0, 255)
    
    return x

In [7]:
def fit(model, save_dir, train_set, test_set, forget_set):
    
    os.makedirs(save_dir, exist_ok=True)
    
    train_x, train_y = train_set.tensors
    forget_x, forget_y = forget_set.tensors
    
    forget_x = add_random_noise(forget_x)
    
    train_set = torch.utils.data.TensorDataset(
        torch.concat([train_x, forget_x], dim=0), 
        torch.concat([train_y, forget_y], dim=0), 
    )
    
    optimizer = torch.optim.Adam(model.parameters())
    error = torch.nn.CrossEntropyLoss()
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):
        
        # train
        
        accs = list()
        
        train_time = 0
        
        model.train()
        
        start_time = time.time()
        
        for x, y in train_loader:
            optimizer.zero_grad()
            output = model(x.cuda())
            y = y.cuda()
            loss = error(output, y)
            loss.backward()
            optimizer.step()
            
            train_time += time.time() - start_time
            
            predicted = torch.argmax(output.data, dim=-1)
            accs.append((predicted == y).float().mean().detach().cpu().numpy())
            
            start_time = time.time()
            
        train_times.append(train_time)
        train_accs.append(np.mean(accs))
        
        # test
            
        model.eval()
        with torch.no_grad():
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            

            x, y = forget_set.tensors

            for i in range(0, x.shape[0], BATCH_SIZE):

                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

            forget_accs.append(np.mean(accs))
        
        # save
        torch.save(model.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))

    return train_times, train_accs, test_accs, forget_accs

In [8]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/Unlearnable/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [9]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/Unlearnable.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1       12.197492   0.862650  0.975739    0.975904
           2       10.109900   0.940800  0.981330    0.981457
           3       10.314586   0.948533  0.984026    0.983904
           4       10.177313   0.954917  0.985523    0.985599
           5       10.217436   0.958850  0.985623    0.985410
10         1        9.943733   0.867033  0.978934    0.979042
           2       10.187575   0.946133  0.981729    0.982036
           3       10.533017   0.949667  0.985224    0.984780
           4       10.228199   0.953500  0.986422    0.986589
           5       10.616878   0.956467  0.986022    0.986776
20         1       10.002365   0.874450  0.978335    0.976517
           2       10.004757   0.945867  0.982728    0.981786
           3       10.251070   0.951350  0.984325    0.984012
           4       10.223609   0.955633  0.987121    0.985374
           5       10.217983   0.958617  0.986122    0.986192
30         1       10.159244   0.866150  0.976538    0.975385
           2       10.183870   0.941867  0.983227    0.980879
           3       10.238254   0.951067  0.982628    0.981485
           4       10.343989   0.954683  0.983427    0.984339
           5       10.358709   0.956150  0.986621    0.986765
40         1       10.083722   0.872117  0.980731    0.979598
           2       10.389214   0.946400  0.979633    0.980303
           3       10.279586   0.952917  0.984125    0.984360
           4       10.659853   0.954900  0.983626    0.984978
           5       10.253969   0.956783  0.986821    0.987888
50         1        9.902655   0.844583  0.976637    0.975370
           2       10.267082   0.940717  0.978934    0.977443
           3       10.229579   0.948850  0.981929    0.983064
           4       10.100196   0.951517  0.985723    0.984787
           5       10.076050   0.954100  0.986522    0.985062
60         1       10.155516   0.825833  0.973842    0.973705
           2       10.382768   0.939650  0.979233    0.979290
           3        9.299946   0.948783  0.983127    0.983897
           4       10.135671   0.953433  0.986322    0.986244
           5       10.516332   0.956917  0.983626    0.984853
70         1        9.999681   0.838617  0.975639    0.976572
           2       10.332960   0.938467  0.982927    0.981454
           3       10.460882   0.948567  0.982728    0.981877
           4       10.524399   0.953100  0.985423    0.984567
           5       10.012012   0.957217  0.985823    0.986393
80         1       10.238461   0.832050  0.978135    0.976506
           2       10.353207   0.940483  0.981430    0.980281
           3       10.259863   0.947833  0.982428    0.980385
           4       10.131404   0.952500  0.984325    0.983005
           5       10.177033   0.953983  0.986022    0.985918
90         1       10.493388   0.864467  0.978035    0.976605
           2       10.580061   0.943100  0.983427    0.981712
           3        9.973269   0.950000  0.983327    0.983087
           4       10.398201   0.955500  0.983926    0.982634
           5       10.413992   0.957367  0.987720    0.986397
99         1       10.126053   0.870267  0.976737    0.975432
           2       10.314359   0.941783  0.982129    0.982258
           3       10.465690   0.950117  0.983726    0.983180
           4        9.915742   0.953733  0.986621    0.986708
           5       10.070861   0.958783  0.988818    0.987702